In [ ]:
import os
import glob
import numpy as np

sacred_dir = "logs"
parent_dir = os.getcwd()
print(os.getcwd())
test_json_files = glob.glob(
    os.path.join(parent_dir, sacred_dir, "**/test**/*info.json"), recursive=True)
print(len(test_json_files))

In [ ]:
import json


def exp_name_from_file(file):

    exp_name = os.path.basename(os.path.dirname(file))
    exp_name = exp_name.split("_")[2:]

    if 'owf' in exp_name and 'dqn' in exp_name:
        index_env = 5
    elif "owf" in exp_name:
        index_env = 4
    else:
        index_env = 5

    if exp_name[index_env] != exp_name[2] and index_env is not 4:
        if int(exp_name[2]) == 10 and int(exp_name[index_env]) < 10:
            pass
        else:
            if not "dqn" in exp_name:
                print("merde")
                print(int(exp_name[2]) == 10, int(exp_name[index_env]) < 10)
                print(exp_name, exp_name[2], exp_name[index_env])
    if "cc_True" in file:
        exp_name = [exp_name[0], exp_name[index_env - 1], exp_name[index_env],
                    "cc_True"]
    else:
        exp_name = [exp_name[0], exp_name[index_env - 1], exp_name[index_env],
                    "cc_False"]
    if exp_name[1] == "sarl":
        exp_name[1] = "owf"
    return exp_name


def dict_results_from_json(test_json_files, method, env1, env2, cc):
    dict_all = []

    for i in test_json_files:
        exp_name = exp_name_from_file(i)

        if cc and "cc_True" in i:
            pass
        elif not cc and ("cc_False" in i or "cc" not in i):
            pass
        else:
            continue

        if exp_name[0] == method and exp_name[1] == env1 and exp_name[
            2] == env2:
            exp_name = "_".join(exp_name)

            # Remove dqn too much
            if exp_name.startswith("dqn"):
                # ignore uc/ic 10 50 100 and owf 5 25 50
                if "10" in k or "50" in k or "100" in k or (
                        "owf" in k and ('5' in k or '25' in k or '50' in k)):
                    continue
        else:
            continue
        try:
            with open(i, "r") as file:
                infos = json.load(file)
                dict_exp = {}
                dict_exp["name"] = os.path.basename(os.path.dirname(i))
                dict_exp["max_test_return_mean"] = np.max(
                    [i["value"] for i in infos["test_return_mean"][:101]])
                dict_exp["last_test_return_mean"] = \
                infos["test_return_mean"][100]["value"]
                #dict_exp["test_return_mean_T"] = infos["test_return_mean_T"][100]
                dict_all.append(dict_exp)
        except IndexError:
            print("index error", len(infos["test_return_mean"]),
                  len(infos["test_return_mean_T"]),
                  infos["test_return_mean"][100],
                  infos["test_return_mean_T"][100])
        except Exception as e:
            print("Exception", i, e)
    return dict_all


methods = ["qmix", "qvmix", "coma", "iql", "qplex", "dqn", "facmac"]
env1 = ["uc", "c", "owf"]
env2 = {"uc": ["3", "5", "10", "50", "100"],
        "c": ["3", "5", "10", "50", "100"], "owf": ["1", "2", "5", "25", "50"]}
cc = [True, False]
results = {}
for cc_ in cc:
    for i in methods:
        for j in env1:
            for k in env2[j]:
                # if dqn and ((uc or c) with 10 50 100) or owf with (5 25 50), skip
                if i == "dqn" and (
                        (j in ["uc", "c"] and k in ["10", "50", "100"]) or (
                        j == "owf" and k in ["5", "25", "50"])):
                    continue
                exp_name = f"{i}_{j}_{k}_cc_{cc_}"
                results[exp_name] = {
                    "experiments": dict_results_from_json(test_json_files, i,
                                                          j, k, cc_)}

# compute median and perc for each exp
# For each exp, we keep the max result
for k, v in results.items():
    if len(v) == 0:
        print("EMPTY", k)
        continue
    list_max_return = [i["max_test_return_mean"] for i in v["experiments"]]
    # print(k, len(list_return), list_return[0])
    np_ = np.array(list_max_return)
    results[k]["list_max_return"] = np_
    median = np.median(np_, axis=0)
    perc_25 = np.percentile(np_, 25, axis=0)
    perc_75 = np.percentile(np_, 75, axis=0)
    min = np.min(np_, axis=0)
    max = np.max(np_, axis=0)
    results[k]["max_median"] = median
    results[k]["max_perc_25"] = perc_25
    results[k]["max_perc_75"] = perc_75
    results[k]["max_min"] = min
    results[k]["max_max"] = max

    # list_last_return = [i["last_test_return_mean"] for i in v["experiments"]]
    # np_ = np.array(list_last_return)
    # median = np.median(np_, axis=0)
    # perc_25 = np.percentile(np_, 40, axis=0)
    # perc_75 = np.percentile(np_, 60, axis=0)
    # min = np.min(np_, axis=0)
    # max= np.max(np_, axis=0)
    # results[k]["last_median"]=median
    # results[k]["last_perc_25"]=perc_25
    # results[k]["last_perc_75"]=perc_75
    # results[k]["last_min"]=min
    # results[k]["last_max"]=max

In [ ]:
color_dict = {
    "qmix": "blue",
    "qvmix": "green",
    "coma": "cyan",
    "iql": "orange",
    "qplex": "red",
    "dqn": "black",
    "facmac": "violet",
    "heuristic":"brown"
}
color_dict_legend={
    "QMIX": "blue",
    "QVMix": "green",
    "QPLEX": "red",
    "COMA": "cyan",
    "FACMAC": "violet",
    "IQL": "orange",
    "DQN": "black",
    "Heuristic":"brown",
}
#Method & uc3 & uc5 & uc10 & uc50 & uc100 & c3 & c5 & c10 & c50 & c100 & owf1 & owf2 & owf5 & owf25 & owf50 \\
#heuristic & -13.0 & 27.5 & -68.8 & -268.2 & -261.6 & -12.6 & -25.4 & -68.5 & -240.9 & -219.9 & -58.4 & -117.1 & -293.1 & -1462.5 & -2925.6\\
# "uc_3": -13.0,
dict_heuristic={'heuristics_10_9uc_ref_2023_04_15_130930.npz': -63.702031546300006,
 'heuristics_5_4uc_camp_2023_04_15_130400.npz': -28.56832624025458,
 'heuristics_10_9uc_camp_2023_04_15_130400.npz': -64.54566487487939,
 'heuristics_3_2c_camp_2023_04_15_130700.npz': -15.244287768169668,
 'heuristics_100_95uc_camp_2023_04_15_130330.npz': -231.51831502811925,
 'heuristics_3_2uc_ref_2023_04_15_130930.npz': -12.548031501250763,
 'heuristics_100_95c_camp_2023_04_15_130630.npz': -194.00829458241884,
 'heuristics_5_4uc_ref_2023_04_15_130930.npz': -25.16981083470593,
 'heuristics_50_48c_ref_2023_04_15_130801.npz': -239.99827314977028,
 'heuristics_50_48c_camp_2023_04_15_130630.npz': -211.0397273727547,
 'heuristics_50_48uc_camp_2023_04_15_130330.npz': -232.66864066644195,
 'heuristics_5_4c_ref_2023_04_15_130801.npz': -28.089348983294887,
 'heuristics_100_95uc_ref_2023_04_15_130930.npz': -262.437213500905,
 'heuristics_5_4c_camp_2023_04_15_130630.npz': -30.46859501673695,
 'heuristics_3_2uc_camp_2023_04_15_130400.npz': -15.096777503160995,
 'heuristics_10_9c_camp_2023_04_15_130630.npz': -68.45376456917336,
 'heuristics_3_2c_ref_2023_04_15_130801.npz': -12.997522667776952,
 'heuristics_10_9c_ref_2023_04_15_130801.npz': -67.70872642438087,
 'heuristics_100_95c_ref_2023_04_15_130801.npz': -218.08651217477365,
 'heuristics_50_48uc_ref_2023_04_15_130930.npz': -268.06588862167297,
 'heuristics_owf_2_3camp_2023_04_15_130030.npz': -115.20353634573738,
 'heuristics_owf_1_3ref_2023_04_15_131101.npz': -58.30502638737299,
 'heuristics_owf_25_3camp_2023_04_15_125129.npz': -1248.1854220407279,
 'heuristics_owf_5_3ref_2023_04_15_131101.npz': -292.2865772354046,
 'heuristics_owf_2_3ref_2023_04_15_131101.npz': -116.91581627560167,
 'heuristics_owf_5_3camp_2023_04_15_125159.npz': -267.2232210135788,
 'heuristics_owf_50_3camp_2023_04_15_125029.npz': -2436.2737196188314,
 'heuristics_owf_50_3ref_2023_04_15_131100.npz': -2925.0128544559384,
 'heuristics_owf_1_3camp_2023_04_15_125559.npz': -62.15511188227698,
 'heuristics_owf_25_3ref_2023_04_15_131101.npz': -1463.7829664424578}

heuristic_results_cc_False = {
    # uc ref
    "uc_3":  -12.548031501250763,
    "uc_5":  -25.16981083470593,
    "uc_10": -63.702031546300006,
    "uc_50": -268.06588862167297,
    "uc_100": -262.437213500905,
    # c ref
    "c_3": -12.997522667776952,
    "c_5": -28.089348983294887,
    "c_10": -67.70872642438087,
    "c_50": -239.99827314977028,
    "c_100": -218.08651217477365,
    # owf ref
    "owf_1": -58.30502638737299,
    "owf_2": -116.91581627560167,
    "owf_5": -292.2865772354046,
    "owf_25": -1463.7829664424578,
    "owf_50": -2925.0128544559384
}
heuristic_results_cc_True = {
    # uc camp
    "uc_3": -15.096777503160995,
    "uc_5": -28.56832624025458,
    "uc_10": -64.54566487487939,
    "uc_50": -232.66864066644195,
    "uc_100": -231.51831502811925,
    # c camp
    "c_3": -15.244287768169668,
    "c_5": -30.46859501673695,
    "c_10": -68.45376456917336,
    "c_50": -211.0397273727547,
    "c_100": -194.00829458241884,
    # owf camp
    "owf_1": -62.15511188227698,
    "owf_2": -115.20353634573738,
    "owf_5": -267.2232210135788,
    "owf_25": -1248.1854220407279,
    "owf_50": -2436.2737196188314
}

# Compare dict_heuristic and heuristic_results_cc_False
# To do so, start with heuristic_results_cc_False and, for each VALUE, print the key in heuristic_results_cc_False and the key that has this value in dict_heuristic
for key, value in heuristic_results_cc_False.items():
    for key2, value2 in dict_heuristic.items():
        if value == value2:
            print(key, key2)
print("  ")
for key, value in heuristic_results_cc_True.items():
    for key2, value2 in dict_heuristic.items():
        if value == value2:
            print(key, key2)

In [ ]:
def data_needed(method, env, cc, limit=None):
    #env2 = {"uc":["3", "5", "10", "50", "100" ], "c":["3", "5", "10", "50", "100"], "owf":["1","2","5","25","50"]}

    if env == "c" or env == "uc":
        _3 = results[f'{method}_{env}_3_cc_{cc}']
        _5 = results[f'{method}_{env}_5_cc_{cc}']
        if method is not "dqn":
            _10 = results[f'{method}_{env}_10_cc_{cc}']
            _50 = results[f'{method}_{env}_50_cc_{cc}']
            _100 = results[f'{method}_{env}_100_cc_{cc}']
        if not cc:
            heuristic_3 = heuristic_results_cc_False[f'{env}_3']
            heuristic_5 = heuristic_results_cc_False[f'{env}_5']
            if method is not "dqn":
                heuristic_10 = heuristic_results_cc_False[f'{env}_10']
                heuristic_50 = heuristic_results_cc_False[f'{env}_50']
                heuristic_100 = heuristic_results_cc_False[f'{env}_100']
        else:
            heuristic_3 = heuristic_results_cc_True[f'{env}_3']
            heuristic_5 = heuristic_results_cc_True[f'{env}_5']
            if method is not "dqn":
                heuristic_10 = heuristic_results_cc_True[f'{env}_10']
                heuristic_50 = heuristic_results_cc_True[f'{env}_50']
                heuristic_100 = heuristic_results_cc_True[f'{env}_100']
    else:
        _3 = results[f'{method}_{env}_1_cc_{cc}']
        _5 = results[f'{method}_{env}_2_cc_{cc}']
        if method is not "dqn":
            _10 = results[f'{method}_{env}_5_cc_{cc}']
            _50 = results[f'{method}_{env}_25_cc_{cc}']
            _100 = results[f'{method}_{env}_50_cc_{cc}']
        if not cc:
            heuristic_3 = heuristic_results_cc_False[f'{env}_1']
            heuristic_5 = heuristic_results_cc_False[f'{env}_2']
            if method is not "dqn":
                heuristic_10 = heuristic_results_cc_False[f'{env}_5']
                heuristic_50 = heuristic_results_cc_False[f'{env}_25']
                heuristic_100 = heuristic_results_cc_False[f'{env}_50']
        else:
            heuristic_3 = heuristic_results_cc_True[f'{env}_1']
            heuristic_5 = heuristic_results_cc_True[f'{env}_2']
            if method is not "dqn":
                heuristic_10 = heuristic_results_cc_True[f'{env}_5']
                heuristic_50 = heuristic_results_cc_True[f'{env}_25']
                heuristic_100 = heuristic_results_cc_True[f'{env}_50']

    # 10 max return per experiments
    if method is not "dqn":
        returns = np.array([_3["list_max_return"], _5["list_max_return"],
                            _10["list_max_return"], _50["list_max_return"],
                            _100["list_max_return"]], dtype=float)
        maxs = np.array(
            [_3["max_max"], _5["max_max"], _10["max_max"], _50["max_max"],
             _100["max_max"]], dtype=float)
        mins = np.array(
            [_3["max_min"], _5["max_min"], _10["max_min"], _50["max_min"],
             _100["max_min"]], dtype=float)
        heuristics = np.array(
            [heuristic_3, heuristic_5, heuristic_10, heuristic_50,
             heuristic_100], dtype=float)
    else:
        returns = np.array([_3["list_max_return"], _5["list_max_return"]], dtype=float)
        maxs = np.array([_3["max_max"], _5["max_max"]], dtype=float)
        mins = np.array([_3["max_min"], _5["max_min"]], dtype=float)
        heuristics = np.array([heuristic_3, heuristic_5], dtype=float)

    # We want how much % it is better then heuristic
    maxs = [(max_ - heuristic) / abs(heuristic) for heuristic, max_ in
            zip(heuristics, maxs)]
    mins = [(min_ - heuristic) / abs(heuristic) for heuristic, min_ in
            zip(heuristics, mins)]
    returns = [(return_ - heuristic) / abs(heuristic) for heuristic, return_ in
               zip(heuristics, returns)]
    if limit is not None:
        returns = [np.clip(i, limit, 100) for i in returns]
        maxs = np.clip(maxs, limit, 100)
        mins = np.clip(mins, limit, 100)

    return returns, maxs, mins, heuristics

returns, maxs, mins, heuristics = data_needed("qmix", "uc", True, limit=-1.01)



In [ ]:
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import matplotlib.patches as patches

list_method = ["dqn", "iql", "facmac", "coma", "qplex", "qvmix", "qmix"]
list_env = [("uc",False), ("c",False), ("owf",False), ("uc",True), ("c",True), ("owf",True)]
n_agents={"uc": [3,5,10,50,100], "c":[3,5,10,50,100], "owf":[2,4,10,50,100]}
cc = False
env_to_title={"uc":"uncorrelated k-out-of-n", "c":"correlated k-out-of-n", "owf":"offshore wind farm"}
cc_to_title={True:"with campaign cost", False:"without campaign cost"}
plt.style.use('seaborn-white')
fig, axes = plt.subplots(2, 3)
fig.set_figheight(80)
fig.set_figwidth(60)

for ax_id, (env,cc) in enumerate(list_env):
    #ax_id = 0,1,..,6
    # ax_ = axes[i][j]
    ax_ = axes[ax_id//3][ax_id%3]
    for idx, method in enumerate(list_method):
        returns, maxs, mins, heuristics = data_needed(method, env, cc, limit=-1.001)
        #print("maxmax", np.max(maxs))
        if method is not "dqn":
            positions = [1.5 * idx + 1.2 - .9, 1.5 * idx + .9 - .9 , 1.5 * idx + .6 - .9,
                         1.5 * idx + .3 - .9, 1.5 * idx + 0 - .9]
        else:
            positions = [0.3, 0]
        c = color_dict[method]
        bp=ax_.boxplot(returns,
                    positions=positions,
                    whis=0,
                    widths=0.29,
                    vert=False,
                    showfliers=False,
                    patch_artist=True,
                    boxprops=dict(facecolor=c, color=c, alpha=0.5),
                    capprops=dict(color=c, alpha=0),
                    whiskerprops=dict(color=c, alpha=0),
                    flierprops=dict(color=c, markeredgecolor=c, alpha=0),
                    medianprops=dict(color=c, linewidth=8))

        for i, j in zip(maxs, positions):
            ax_.plot(i, j, "o", color=c, markersize=10)

        for i, j in zip(mins, positions):
            ax_.plot(i, j, "o", color=c, markersize=10)
    #print(" ")
    # set x limit

    # ax_.set_xscale('symlog', base=10, linthresh=1)

    xticks = [-1, -0.5, 0, 0.5]
    ax_.set_xticks(xticks)
    def float_to_perc(number):
        if number == 0:
            return "0"
        elif number == 0.25:
            return "25%"
        elif number == 0.75:
            return "75%"
        elif number == 0.5:
            return "50%"
        elif number == -0.5:
            return "-50%"
        elif number == -1:
            return "-100%"
        elif number == -10:
            return "-1000%"
        else:
            return ""

    ax_.set_xticklabels([float_to_perc(x) for x in xticks], fontsize=50)
    # if ax_idx%2 !=0:
    #     labels = ax_.get_xticklabels()
    #     for label in labels:
    #         label.set_y(label.get_position()[1] + 0.04)  # adjust the y-coordinate here


    ax_.spines['left'].set_position(('data', 0))
    ax_.spines['left'].set_color("brown")
    ax_.spines['left'].set_linewidth(5)

    ax_.spines['top'].set_visible(False)
    ax_.spines['right'].set_visible(False)
    ax_.spines['bottom'].set_visible(True)

    ax_.set_yticklabels([])

    if cc:
        ax_.set_title(env_to_title[env]+"; campaign cost", fontsize=50)
    else:
        ax_.set_title(env_to_title[env], fontsize=50)

    ax_.xaxis.set_tick_params(length=15, width=5)
    # ax_.plot((-1,-1),(-1,10), color="black", linestyle="--")
    fig.canvas.draw()
    # Create a sample plot


    data = [[i, j] for i, j in zip(n_agents[env], heuristics)]

    # Add the table
    table_text = '     H | n\n'
    for row in data:
        table_text += f'{row[1]:7.1f}| {row[0]:<3}\n'
    text=ax_.text(0.05, 0.83, table_text, transform=ax_.transAxes, fontsize=42, family='monospace')

    if ax_id==0:
        # qmix 1
        x1, y1 = [0.3, 0.67], [0.951, .951]
        ax_.plot(x1, y1, transform=ax_.transAxes, color="blue", linewidth=3, linestyle="--")
        # qvmix 1
        x1, y1 = [0.3,0.4, 0.4, 0.67], [0.949, 0.949, .805, .805]
        ax_.plot(x1, y1, transform=ax_.transAxes, color="green", linewidth=3, linestyle="--")

        x1, y1 = [0.06, 0.05, 0.05, 0.48], [0.859, 0.859, 0.835, 0.835]
        ax_.plot(x1, y1, transform=ax_.transAxes, color="blue", linewidth=3, linestyle="--")

        x1, y1 = [0.06, 0.05, 0.05, 0.3], [0.859, 0.859, 0.685, 0.685]
        ax_.plot(x1, y1, transform=ax_.transAxes, color="green", linewidth=3, linestyle="--")


handles = []
for label, color in color_dict_legend.items():
    handle = Patch(color=color, label=label)
    handles.append(handle)

legend=fig.legend(handles=handles, loc="lower center", ncol=8, prop={'size': 50}, bbox_to_anchor=(0.5, 0.06))
for line, text in zip(legend.get_lines(), legend.get_texts()):
    text.set_color(line.get_color())

fig.canvas.draw()

limits=[(-1.001, .75), (-1.001, .75), (-1.001, .75), (-1.001, .75), (-1.001, .75), (-1.001, .75)]
for idx, ax_ in enumerate(axes):
    for idx2, ax2_ in enumerate(ax_):
        ax2_.set_xlim(limits[idx])

for idx, ax_ in enumerate(axes):
    ax_[1].set_xlabel("Normalised relative reward (x-H)/H", labelpad=40, fontsize=50)


fig.canvas.draw()
plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.13)
plt.plot()
plt.savefig(f"sacred_scripts/plots/boxplot_perc_limit.pdf",
            bbox_inches='tight')
# print height width of the fig
print(fig.get_figheight(), fig.get_figwidth())